In [4]:
!pip install transformers sentence-transformers datasets rouge-score streamlit evaluate

In [5]:
from sentence_transformers import SentenceTransformer
from transformers import pipeline
import evaluate # Import evaluate directly
import numpy as np
import streamlit as st  # For optional UI later

In [6]:
dataset= []

with open('/content/LangChain_Bot_dataset.txt', 'r') as file:
  dataset= file.readlines()
  print(f'Loaded {len(dataset)} lines from file.')

Loaded 50 lines from file.


In [7]:
Embedding_model= 'all-MiniLM-L6-v2'
embedder= SentenceTransformer(Embedding_model)

Vector_DB= []
for chunk in dataset:
  embedding = embedder.encode([chunk])[0]
  Vector_DB.append((chunk, embedding))
  print(f'Add chunks to DB (shape: {embedding.shape})')

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks to DB (shape: (384,))
Add chunks

In [8]:
def cosine_similarity(a, b):
  dot_product = np.dot(a, b)
  norm_a= np.linalg.norm(a)
  norm_b= np.linalg.norm(b)
  return dot_product / (norm_a * norm_b)

In [9]:
def retrieve(query, top_n= 3):
  query_embedding= embedder.encode([query])[0]
  similarities= []
  for chunk, embedding in Vector_DB:
    sim= cosine_similarity(query_embedding, embedding)
    similarities.append([chunk, sim])

  similarities.sort(key= lambda x: x[1], reverse= True)
  return similarities[:top_n]

In [10]:
Generation_model = 'gpt2'
generator = pipeline('text-generation', model= Generation_model)

def generate_ans(query, retrieved_knowledge):
  context = '\n'.join([f'- {chunk}' for chunk, _ in retrieved_knowledge])
  prompt= f'''You are a helpful LangChain documentation assistant.
  Use ONLY the following context to answer the technical question. Be concise, accurate, and code-focused for developers.
  context: {context}
  Question: {query}
  Answer:'''

  response = generator(prompt, max_new_tokens= 50, num_return_sequences= 1, do_sample= True, temperature= 0.1)[0]['generated_text']
  answer = response.split('Answer:')[-1].strip()
  return answer

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [11]:
input_query= input('Ask any Langchain doc Question:')
retrieved= retrieve(input_query)
print('Retrieved Info: ')
for chunk, sim in retrieved:
  print(f'- (similarity {sim:.2f}) {chunk}')

generated_ans= generate_ans(input_query, retrieved)
print('\nGenerated Answer: ', generated_ans)

Ask any Langchain doc Question:what is lcel?


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Retrieved Info: 
- (similarity 0.55) LCEL (LangChain Expression Language) offers a declarative and readable syntax for linking multiple components together.

- (similarity 0.55) LCEL improves readability and maintainability of complex pipeline definitions.

- (similarity 0.23) Prompt engineering remains central to improving accuracy, consistency, and interpretability of chain outputs.


Generated Answer:  lcel is a language that provides a declarative syntax for linking multiple components together. It is a declarative syntax that provides a declarative syntax for linking multiple components together. It is a declarative syntax that provides a declar


In [16]:
rouge = evaluate.load('rouge')
test_queries = [
    ("What is LCEL in LangChain?", "LCEL is LangChain Expression Language for composing chains declaratively."),
    ("How do agents work?", "Agents use LLMs for reasoning and tools for actions like ReAct."),
    ("Explain retrieval in LangChain.", "Retrieval enables RAG by embedding and fetching relevant doc chunks."),
    ("What are chains in LangChain?", "Chains are sequences of calls to LLMs or tools combined for a task."),
    ("What is a retriever?", "A retriever is an interface that returns relevant documents given a query."),
    ("How does memory work in LangChain?", "Memory lets chains and agents persist state across interactions."),
    ("What is RAG?", "RAG stands for Retrieval-Augmented Generation, combining retrieval with LLM outputs."),
    ("What are tools in LangChain?", "Tools are external functions or APIs that agents can call to take actions."),
    ("What is LangSmith?", "LangSmith is a platform for debugging, testing, and monitoring LangChain apps."),
    ("How do callbacks work?", "Callbacks let you log, stream, or monitor events during chain or agent execution.")
]

scores = []
for q, ref in test_queries:
    ret = retrieve(q)
    ans = generate_ans(q, ret)
    score = rouge.compute(predictions=[ans], references=[ref])['rougeL']
    scores.append(score)
print(f'Avg ROUGE-L Score: {np.mean(scores):.2f}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Avg ROUGE-L Score: 0.10


In [ ]:
def streamlit_app():
    st.title('RAG LangChain DocBot')
    query = st.text_input('Ask about LangChain:')
    if query:
        ret = retrieve(query)
        ans = generate_ans(query, ret)
        st.write('Retrieved:', [c for c,_ in ret])
        st.write('Answer:', ans)

if __name__ == '__main__':
    streamlit_app()